# Réseau convolutif

Objectif : construction d'un réseau convolutif simple avec Keras.
De nombreuses possibilités d'amélioration :
- optimisation du nombre de couches / noyaux / activations
- architecture
- régularisation
- ...

# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow.keras as tk
from tensorflow.keras.datasets import mnist

from tensorflow.keras.layers import Input, Dense, Activation,Lambda,Flatten,Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import Adam


# Données

On charge ensuite les données MNIST. Les paramètres de la base sont récupérés (nombre d'exemples, de classes, taille de la rétine).

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()


num_examples = x_train.shape[0]
num_test = x_test.shape[0]
num_input = x_train.shape[1]*x_train.shape[2]
num_classes = 10

img_size = x_train.shape[1]
img_shape = (img_size, img_size)

x_train = x_train.reshape(num_examples, img_size, img_size, 1)/255
x_test = x_test.reshape(x_test.shape[0], img_size, img_size, 1)/255

print('Taille de la rétine : ',num_input)
print("Nombre d'exemples : ",num_examples)

# Conversion en matrices binaires
y_train = tk.utils.to_categorical(y_train, num_classes)
y_test = tk.utils.to_categorical(y_test, num_classes)


11490434/11490434 [==============================] - 0s 0us/step
Taille de la rétine :  784
Nombre d'exemples :  60000


# Définition du réseau

## Paramètres

In [3]:
lr = 0.001
num_epochs = 25
batch_size = 128
dropout = 0.75 # ici probabilité de garder le neurone

## Types de couches

## Modèle du réseau construit
CONV1-RELU-MAX POOLING - CONV2-RELU-MAX POOLING-FCL-Dropout-Prediction

In [4]:
#TODO
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1),activation = "relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5),activation = "relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units = 1024,activation = "relu"))
model.add(Dropout(dropout))
model.add(Dense(units = 10,activation = "softmax"))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 64)          51264     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 4, 4, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 1024)              1

## Fonction de coût et optimisation

In [5]:
adam = Adam(learning_rate=lr)
model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['acc'])

# Entraînement du réseau

In [ ]:
hist=model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size,verbose=1)

Epoch 1/25
469/469 [==============================] - 16s 6ms/step - loss: 0.0573 - acc: 0.9010
Epoch 2/25
469/469 [==============================] - 2s 5ms/step - loss: 0.0130 - acc: 0.9822
Epoch 3/25
172/469 [==========>...................] - ETA: 1s - loss: 0.0108 - acc: 0.9850

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
plt.figure(figsize=(15,8))
plt.subplot(1,2,1)
plt.plot(hist.history['acc'])
#plt.plot(hist.history['val_acc'])
plt.title('Précision')
plt.ylabel('Précision')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='best')

plt.subplot(1,2,2)
x = list(range(1,num_epochs+1))
l = hist.history['loss']
plt.xlabel('epoch')
plt.ylabel('Entropie croisée')
plt.title("Test : Score = {0:5.3f}, Précision = {1:5.3f}".format(score[0], score[1]))
plt.plot(x,l)
plt.show()
plt.tight_layout()

In [ ]:
predicted_classes = np.argmax(model.predict(x_test), axis=-1)
y = np.argmax(y_test,axis=1)

incorrects = np.nonzero(predicted_classes != y)[0]
plt.figure(figsize=(10,10))
for i in range(0,9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[incorrects[i]].reshape(28,28), cmap='gray', interpolation='none')
    plt.title( "Prédit/vrai : {}/{}".format(predicted_classes[incorrects[i]], y[incorrects[i]]))
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()
print("Nombre d'erreurs {}/{}\n\n".format(incorrects.size,y.size))
